# Symbolic calculations in Julia

`Symbolics.jl` is a computer Algebra System (CAS) for Julia. The symbols are number-like and follow Julia semantics so we can put them into a regular function to get a symbolic counterpart.

Source:
- [Simulating Big Models in Julia with ModelingToolkit @ JuliaCon 2021 Workshop](https://youtu.be/HEVOgSLBzWA).
- [Symbolics.jl](https://github.com/JuliaSymbolics/Symbolics.jl) Github repo and its [docs](https://symbolics.juliasymbolics.org/dev/).

## Caveats about Symbolics.jl

1. `Symbolics.jl` can only handle *traceble*, *quasi-static* expressions. However, some expressions are not quasi-static e.g. factorial. The number of operations depends on the input value.

> Use `@register` to make it a primitive function.

2. Some code paths is *untraceable*, such as conditional statements: `if`...`else`...`end`.

> You can use `ifelse(cond, ex1, ex2)` to make it traceable.

## Basic operations
- `latexify`
- `derivative`
- `gradient`
- `jacobian`
- `substitute`
- `simplify`

In [ ]:
using Symbolics
using Latexify
PNG(img) = display("image/png", img) ## Force PNG output

In [ ]:
@variables x y

x^2 + y^2

You can use Latexify to see the LaTeX code.

In [ ]:
A = [x^2 + y 0 2x
     0       0 2y
     y^2 + x 0 0]

latexify(A)

Derivative: `Symbolics.derivative(expr, variable)`

In [ ]:
Symbolics.derivative(x^2 + y^2, x)

Gradient: `Symbolics.gradient(expr, [variables])`

In [ ]:
Symbolics.gradient(x^2 + y^2, [x, y])

Jacobian: `Symbolics.jacobian([exprs], [variables])`

In [ ]:
Symbolics.jacobian([x^2 + y^2; y^2], [x, y])

Substitute: `Symbolics.substitute(expr, mapping)`

In [ ]:
Symbolics.substitute(sin(x)^2 + 2 + cos(x)^2, Dict(x=>y^2))

In [ ]:
Symbolics.substitute(sin(x)^2 + 2 + cos(x)^2, Dict(x=>1.0))

Simplify: `Symbolics.simplify(expr)`

In [ ]:
Symbolics.simplify(sin(x)^2 + 2 + cos(x)^2)

This expresseion gets automatically simplified because it's always true

In [ ]:
2x - x

In [ ]:
ex = x^2 + y^2 + sin(x)
isequal(2ex, ex + ex)

In [ ]:
ex / ex

Symblic integration: use [SymbolicNumericIntegration.jl](https://github.com/SciML/SymbolicNumericIntegration.jl). The [Youtube video](https://youtu.be/L47k2zjPU9s) by `doggo dot jl` gives a concise example.

## Custom functions used in Symbolics

With `@register`, the `rand()` in `foo()` will be evaulated as-is and **will not** be expanded by `Symbolics.jl`.

> By default, new functions are traced to the primitives and the symbolic expressions are written on the primitives. However, we can expand the allowed primitives by registering new functions.

## More number types
Complex number

In [ ]:
@variables z::Complex

Array types with subscript

In [ ]:
@variables xs[1:18]

In [ ]:
xs[1]

Explicit vector form

In [ ]:
collect(xs)

## Example: Rosenbrock function
Wikipedia: https://en.wikipedia.org/wiki/Rosenbrock_function
We use the vector form of Rosenbrock function

In [ ]:
rosenbrock(xs) = sum( 1:length(xs)-1) do i
    100*(xs[i+1] - xs[i]^2)^2 + (1 - xs[i])^2
end

Minimum when xs are all one's

In [ ]:
rosenbrock(ones(100))

In [ ]:
N = 100
@variables xs[1:N]

A long list of vector components

In [ ]:
xs = collect(xs)

In [ ]:
rxs = rosenbrock(xs)

Gradient

In [ ]:
grad = Symbolics.gradient(rxs, xs)

Hessian = Jacobian of gradient

In [ ]:
hes1 = Symbolics.jacobian(grad, xs)

call `hessian()` directly

In [ ]:
hes2 = Symbolics.hessian(rxs, xs)

In [ ]:
isequal(hes1, hes2)

### Sparse matrix
Sparse Hessian matrix of the Hessian matrix of the Rosenbrock function w.r.t. to vector components. (298 non-zero values out of 10000 elements)

In [ ]:
hes_sp = Symbolics.hessian_sparsity(rxs, xs)

Visualize the sparse matrix with `Plots.spy()`

In [ ]:
using Plots
fig = spy(hes_sp)
fig |> PNG

## Generate functions

- `build_function(ex, args...)` generates out-of-place (oop) and in-place (ip) function expressions in a pair.
- `build_function(ex, args..., parallel=Symbolics.MultithreadedForm())` generates a parallel algorithm to evaluate the output. See the [example](https://symbolics.juliasymbolics.org/stable/tutorials/auto_parallel/) in the official docs.
- `build_function(ex, args..., target=Symbolics.CTarget())` generates a C function from Julia.

In [ ]:
fexprs = build_function(grad, xs);

Extract the Out-of-place version

In [ ]:
foop = eval(fexprs[1])

Extract the In-place version

In [ ]:
fip = eval(fexprs[2])

In [ ]:
inxs = rand(N)
out = similar(inxs)
fip(out, inxs)  ## The inplace version returns nothing. The results are stored in out parameter.

In [ ]:
foop(inxs)

In [ ]:
isapprox(foop(inxs), out)

You can save the generated function for later use: `write("function.jl", string(fexprs[2]))`
`ForwardDiff.jl` checks if our gradient from `Symbolics.jl` is correct.

In [ ]:
using ForwardDiff
ForwardDiff.gradient(rosenbrock, inxs) ≈ out

Sparce Hessian matrix, only non-zero expressions are calculated.

In [ ]:
hexprs = build_function(hes_sp, xs)

In [ ]:
hoop = eval(hexprs[1])
hip = eval(hexprs[2])
hoop(rand(N))

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*